In [1]:
import pandas as pd
import os
import torch
import numpy as np
import random

In [2]:
from Config import *
import TextDataFrame as tdf
import utils

import imp
imp.reload(tdf)
imp.reload(utils)

<module 'utils' from '/home/adrian/Projects/Competition/kaggle-toxic-comments-2021/utils.py'>

In [3]:
utils.set_seed()
df_processor = tdf.DatasetProcessor(DEFAULT_CLEAN_PROCEDURE)

In [4]:
weights_target_dict = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5,  'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}
text_col = 'comment_text'
df_train = pd.read_csv(os.path.join(DATA_PATH,"toxic-comment-classification-challenge/train.csv"))
df_train['y'] = df_processor.set_target(df_train, average_weights_dict = weights_target_dict)
df_train[text_col] = df_processor.clean_text(df_train[text_col])

  0%|          | 0/159571 [00:00<?, ?it/s]

/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Wikipedia:ELYES" looks like a URL. Beautiful Soup is not 

In [19]:
from transformers import (
    AdamW,
    BertConfig,
    BertTokenizer,BertForSequenceClassification)
from keras.preprocessing.sequence import pad_sequences
import torch
from sklearn import preprocessing
import emoji
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
model_dir = "/home/adrian/Projects/Pre-Trained-Models/HateBERT/HateBERT_hateval"

In [48]:
config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load pre-trained Tokenizer from directory, change this to load a tokenizer from ber package
tokenizer = tokenizer_class.from_pretrained(model_dir)

# Load Bert for classification 'container'
model = BertForSequenceClassification.from_pretrained(
    model_dir, # Use pre-trained model from its directory, change this to use a pre-trained model from bert
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

# Set the model to work on CPU if no GPU is present
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
encoded_sent = tokenizer.encode(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100,
            # This function also supports truncation and conversion
            # to pytorch tensors, but we need to do padding, so we
            # can't use these features :( .
            # max_length = 128,          # Truncate all sentences.
            # return_tensors = 'pt',     # Return pytorch tensors.
        )
encoded_sent

[101, 1045, 102]

In [8]:
text_tokenized = df_train[text_col].progress_apply(lambda x:tokenizer.encode(
            x,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100, ))

  0%|          | 0/159571 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:
test_input_ids = pad_sequences(text_tokenized, maxlen=100, dtype="long",
                               value=tokenizer.pad_token_id, truncating="pre", padding="pre")

test_attention_masks = [[int(token_id)>0 for token_id in sent] for sent in test_input_ids]

In [14]:
prediction_inputs, prediction_masks, tweet_ids, tweet_labels = test_input_ids,\
                                                                test_attention_masks,\
                                                                df_train.index,\
                                                                df_train['y']

In [15]:
# Tweets
prediction_inputs = torch.tensor(prediction_inputs)

# Attention masks
prediction_masks = torch.tensor(prediction_masks)

In [18]:
label_encoder = preprocessing.LabelEncoder()
targets = label_encoder.fit_transform(tweet_ids)
# targets: array([0, 1, 2, 3])

prediction_ids = torch.as_tensor(targets)
# targets: tensor([0, 1, 2, 3])

In [116]:
batch_size = 1

In [117]:
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_ids)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [118]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [141]:
predictions = []
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_tweet_ids = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    flat_logits = np.argmax(logits, axis=1).flatten()
    # Get tweet ids for prediction output
    ids = label_encoder.inverse_transform(b_tweet_ids.cpu().numpy())
    # Store predictions and true labels
    predictions.extend(list(zip(ids, flat_logits)))
    break

In [137]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 1
seed = 42

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/2'
# tfhub_handle_encoder = model_dir
def embedding_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, name='BERT_encoder')
    outputs = encoder(encoder_inputs)['pooled_output']
    return tf.keras.Model(text_input, outputs)
embedder = embedding_model()

In [138]:
text_test = ['this is such an amazing movie!']
test_embedding = embedder(tf.constant(text_test))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([ 0.14165208, -0.9998461 , -0.96863383,  0.61972934, -0.06343801,
       -0.99984795,  0.8300238 ,  0.98985773,  0.11514678], dtype=float32)>

In [140]:
from scipy import sparse

<1x256 sparse matrix of type '<class 'numpy.float32'>'
	with 256 stored elements in Compressed Sparse Row format>